In [1]:
import cv2
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import fetch
import os
import keyboard
from tkinter import *
import threading
import glob

In [2]:
from manga_ocr import MangaOcr
mocr=MangaOcr()

2023-05-29 21:30:27.066 | INFO     | manga_ocr.ocr:__init__:13 - Loading OCR model from kha-white/manga-ocr-base
c:\Users\tirth\anaconda3\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
2023-05-29 21:30:37.346 | INFO     | manga_ocr.ocr:__init__:19 - Using CUDA
2023-05-29 21:30:40.257 | INFO     | manga_ocr.ocr:__init__:29 - OCR ready


In [3]:
def listdirs(rootdir,dirs):
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            dirs.append(d)
            listdirs(d,dirs)

current_dir=os.getcwd()
dirs=[]
listdirs(current_dir,dirs)
images = []
for i in dirs:
    images = list(images)+list([os.path.join(i, file)
        for file in os.listdir(i)
        if file.endswith('.png')])

In [23]:
cropping = False
x_start, y_start, x_end, y_end = 0, 0, 0, 0
image = cv2.imread(images[0])
image=cv2.resize(image,(900,780))  ## Extra
oriImage = image.copy()
Destroy=False
Count=1

In [24]:
def Next():
    global Count,image,oriImage
    Count+=1
    if Count>=len(images):
        Count=len(images)-1
    image=cv2.imread(images[Count])
    image=cv2.resize(image,(900,780))  ## Extra
    oriImage=image.copy()
    # print(str(Count)+'.png')

In [25]:
def Prev():
    global Count,image,oriImage
    Count-=1
    if Count<0:
        Count=0
    image=cv2.imread(images[Count])
    image=cv2.resize(image,(900,780))  ## Extra
    oriImage=image.copy()
    # print(str(Count)+'.png')

In [7]:
def mouse_crop(event, x, y, flags, param):
    # grab references to the global variables
    global x_start, y_start, x_end, y_end, cropping,Count

    # if the left mouse button was DOWN, start RECORDING
    # (x, y) coordinates and indicate that cropping is being
    if event == cv2.EVENT_LBUTTONDOWN:
        x_start, y_start, x_end, y_end = x, y, x, y
        cropping = True
    # Mouse is Moving
    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping == True:
            x_end, y_end = x, y
    # if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates
        x_end, y_end = x, y
        cropping = False # cropping is finished
        refPoint = [(x_start, y_start), (x_end, y_end)]
        if len(refPoint) == 2: #when two points were found
            roi = oriImage[refPoint[0][1]:refPoint[1][1], refPoint[0][0]:refPoint[1][0]]
            cv2.imwrite("Cropped.png",roi)
            s=mocr('cropped.png')
            trans=fetch.fetch_data(s)
            print(s,"---->",trans)
            os.remove('Cropped.png')
            cv2.imshow("cropped", roi)

In [8]:
def dest():
    global Destroy
    Destroy=True

In [9]:
def thread1():
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", mouse_crop)
    while Destroy==False:
        # if Destroy:
        #     break
        i = image.copy()
        if not cropping:
            cv2.imshow("image", image)

        elif cropping:
            cv2.rectangle(i, (x_start, y_start), (x_end, y_end), (255, 0, 0), 2)
            cv2.imshow("image", i)
        
        cv2.waitKey(1)
    cv2.destroyAllWindows()

In [10]:
def thread2():
    root = Tk()
    root.geometry('300x100')
    p = Button(root, text = 'Previous', bd = '5',
                            command = Prev)
    p.pack(side = 'left')

    n=Button(root, text = 'Next', bd = '5',
                            command = Next)
    n.pack(side = 'right')

    b=Button(root, text = 'Destroy', bd = '5',
                            command = lambda:[dest(),root.destroy()])
    b.pack(side = 'top')
    root.mainloop()

In [11]:
# thread2()

In [26]:
if __name__=='__main__':
    global Destroy
    Destroy=False
    t1=threading.Thread(target=thread1)
    t2=threading.Thread(target=thread2)
    t1.start()
    t2.start()


変じゃないよな俺 ----> I'm not weird, am I?
最後の第二に鍛えた。帰還者だ。学園でする ----> Forged in the last second. Returnees. I'll do it at the academy.
俺九条恭弥は ----> I, Kyoya Kujo...
気がついたらそこにいた ----> I found myself there.
ここは．．． ----> This place is ...
どこなんだ．．．？ ----> Where are you ...?
ようこそ勇者よ ----> Welcome, my brave men.
そなたを．．．ずっと待っていたぞ ----> I have been waiting for you... I've been waiting for you.
さぁ ----> Come on.
わしを殺しておくれーー ----> Please kill me!
そして ----> and
こいつと出会ったんだ ----> I met this guy.
二年後 ----> two years later
二年後 ----> two years later
ここが．．．「国立ユグラシア学園」： ----> This is the place... National Yugrasia Academy:
へ．．．変じゃないよな俺．．． ----> Heh ... I'm not weird ...
ここにいるのみんな新入生だろ．．．悪目立ちしたくないんだよなぁ．．． ----> We're all freshmen here ... I don't want to make a bad impression...
くっくっへ．．．いよるひよっこ勇者どもめが！ ----> Kkkkkkkkkkkkkkkkkkkkkkkkkkkkkk... You chickenshit heroes!
あっ ----> Ah!
おいフェリス顔出すなって言ってるだろ！ ----> Hey Ferris, I told you not to show your face!
なんじゃ！ケチケチするでないこの中は退屈なのじゃ？ ----> What is it? Don't be stingy, it'